In [1]:
require 'daru'
require 'daru_plotting_plotly'
require 'hg'

class Numeric
  def mili
    (self * 10 ** (-3)).to_f
  end
  
  def micro
    (self * 10 ** (-6)).to_f
  end
end

Daru.plotting_library = :plotly

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

:plotly

In [2]:
class LeastSquareFitting
  def initialize(xs, ys)
    raise ArgumentError "xs and ys should be same size" if xs.size != ys.size
    @xs = xs
    @ys = ys
  end
  
  def fit
    xy = @xs.zip(@ys).map { |x, y| x * y }.sum
    x = @xs.sum
    y = @ys.sum
    x2 = @xs.sum { |x| x ** 2 }
    n = @xs.size
    
    @a = (n*xy - x*y)/(n*x2 - x**2)
    @b = (x2*y - xy*x)/(n*x2 - x**2)
  end
  
  def sigma_y
    @sigma_y ||= Math.sqrt(1.0/(@xs.size - 2)*(@xs.zip(@ys).map {|x, y| (x * @a + @b - y) ** 2}.sum))
  end
  
  def sigma_a
    x = @xs.sum
    x2 = @xs.sum { |x| x ** 2 }
    @sigma_a ||= sigma_y * Math.sqrt(@xs.size.to_f/(@xs.size.to_f*x2 - x**2))
  end
  
  def sigma_b
    x = @xs.sum
    x2 = @xs.sum { |x| x ** 2 }
    @sigma_b ||= sigma_y * Math.sqrt(x2/(@xs.size.to_f*x2 - x**2))
  end
  
  attr_reader :a, :b
end

class ExpFitting < LeastSquareFitting
  def initialize(xs, ys)
    super(xs, ys.map{ |y| Math.log(y) })
  end
  
  def to_s
    "log y = #{'%1.2e' % @a} x #{ @b.positive? ? '+' : '-' } #{'%1.2e' % @b.abs }"
  end
  
  def points(min, max)
    ys = [min, max].map { |x| Math.exp(@b) * Math.exp( @a * x ) }
    { x: [min, max], y: ys, name: to_s }
  end
end

class LinearFitting < LeastSquareFitting
  def points(min, max)
    ys = [min, max].map { |x| x * @a + @b }
    { x: [min, max], y: ys , name: to_s}
  end
  
  def to_s
    "y = #{'%1.2f' % @a} x + #{'%1.2f' % @b}"
  end
end

:to_s

In [19]:
Daru::DataFrame.from_csv('data/nakagawa/dark_solar_battery.csv').tap do |df|
  df.map_rows! do |row|
    row['v'] = - row['v']
    row['i'] = - row['i']
    row
  end
  
  line = df.filter_rows { |row| row['v'] < 0.5 }
  f = LinearFitting.new(line['v'].to_a, line['i'].to_a)
  f.fit
  df.map_rows! do |row|
    row['i'] = row['i'] - (f.a * row['v'] + f.b)
    row
  end
  df.plot(x: 'v', y: 'i').show
  df.filter_rows{|row| row['v'] >= 0.5}.plot(x: 'v', y: 'i', layout: { yaxis: { type: :log } }).show
end
nil

#<Plotly::Offline::HTML:0x007fd99ac1b878 @id="aea9569a-3ed2-4987-991f-6fbc6184fa9f", @data=[{:x=>[-0.844, -0.815, -0.785, -0.704, -0.542, -0.455, -0.348, -0.275, -0.207, -0.146, -0.121, -0.0903, -0.0783, -0.06559999999999999, -0.022, -0.0007, 0.0003, 0.0158, 0.028300000000000002, 0.0594, 0.07790000000000001, 0.103, 0.115, 0.143, 0.159, 0.184, 0.203, 0.617, 1.218], :y=>[-0.0014918946604163368, -0.0008843481902672447, -0.000286886324595767, 0.0004962607127172126, 0.0009625547873431776, 0.0008851941977904556, 0.0007061415186853832, 0.0005699653918193061, 0.000384212287341316, 0.00026905141420664855, 0.0002469363022662109, 0.00018733894480335383, 0.0001663236910719436, 0.0001382492142062013, 4.256045898207847e-05, -1.2416163911742694e-06, -3.3262208687917686e-06, -3.6637590271863026e-05, -6.269514624208182e-05, -0.00013032634549598604, -0.00017089152833190996, -0.000222015100720109, -0.00024303035445151912, -0.00034539927982480886, -0.0003367529514666889, -0.00038886806340712645, -0.00045047554848185877, -0.001325501802215503, -0.00238634909326362], :type=>:scatter, :mode=>"", :name=>"i"}], @layout={:width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fd99c3cf208 @id="5a9034da-0c08-428e-a414-3f9ecdca4cb1", @data=[{:x=>[0.617, 1.218], :y=>[-0.001325501802215503, -0.00238634909326362], :type=>:scatter, :mode=>"", :name=>"i"}], @layout={:width=>1000, :height=>500, :yaxis=>{:type=>:log}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

# 抵抗？

In [12]:
data = ['itano', 'nakagawa'].map do |member|
  ['dark', '5cm', '15cm', '25cm'].map do |name|
    file_name = "data/#{member}/#{name}_solar_battery_negative.csv"
    if File.exists?(file_name)
      Daru::DataFrame.from_csv(file_name).sort(['v']).to_h.map { |k, v| [k == 'v' ? 'x' : 'y', v.to_a] }.to_h.merge(name: "#{member}_#{name}", mode: 'markers+lines')
    end
  end
end.flatten
Plotly::Plot.new(data: data, layout: { xaxis: { title: '電圧(V)' }, yaxis: { title: '電流(I)' } }).show
nil

#<Plotly::Offline::HTML:0x007fd99c38c958 @id="e8c990e2-de95-49aa-9b94-2b341d38979f", @data=[{:x=>[-10.2, -3.48, -1.712, -1.506, -1.418, -1.186, -0.95, -0.537, -0.307, -0.1555, -0.1237, -0.0857, -0.0523, -0.0412, -0.0334, -0.0162, -0.0097, -0.0003, 0.142, 0.247, 0.383, 0.466, 0.554, 0.57, 0.588, 0.62, 0.626, 0.636], :y=>[-0.082, -0.0274, -0.0135, -0.011800000000000001, -0.0111, -0.0092, -0.0074, -0.004200000000000001, -0.0024, -0.0012, -0.00096, -0.00067, -0.000407, -0.000321, -0.000261, -0.000127, -7.7e-05, -4.4e-06, 0.0011, 0.0019, 0.0031000000000000003, 0.004, 0.0063, 0.0070999999999999995, 0.008, 0.009800000000000001, 0.010199999999999999, 0.0109], :mode=>"markers+lines", :name=>"itano_dark"}, {:x=>[0.34, 0.343, 0.356, 0.357, 0.359, 0.359, 0.362, 0.366, 0.371, 0.375, 0.379, 0.384, 0.387, 0.39, 0.393, 0.422], :y=>[-0.0033799999999999998, -0.0033, -0.00317, -0.0029300000000000003, -0.0024100000000000002, -0.0035600000000000002, -0.0025499999999999997, -0.0022, -0.0019, -0.0016, -0.0013000000000000002, -0.001, -0.0008, -0.0006, -0.0004, 0], :mode=>"markers+lines", :name=>"itano_5cm"}, {:x=>[0.253, 0.302, 0.329, 0.352, 0.372, 0.389, 0.403, 0.409, 0.428], :y=>[-0.0025, -0.002, -0.0017, -0.0014, -0.0011, -0.0008, -0.0005, -0.00035, 0], :mode=>"markers+lines", :name=>"itano_15cm"}, {:x=>[0.173, 0.201, 0.222, 0.246, 0.272, 0.293, 0.316, 0.336, 0.37], :y=>[-0.0017, -0.0015, -0.0013000000000000002, -0.0011, -0.0009000000000000001, -0.0007, -0.0005, -0.0003, 0], :mode=>"markers+lines", :name=>"itano_25cm"}, {:x=>[-1.218, -0.617, -0.203, -0.184, -0.159, -0.143, -0.115, -0.103, -0.07790000000000001, -0.0594, -0.028300000000000002, -0.0158, -0.0003, 0.0007, 0.022, 0.06559999999999999, 0.0783, 0.0903, 0.121, 0.146, 0.207, 0.275, 0.348, 0.455, 0.542, 0.704, 0.785, 0.815, 0.844], :y=>[-0.0099, -0.004900000000000001, -0.0016, -0.00147, -0.00127, -0.0011, -0.00092, -0.00082, -0.000618, -0.000472, -0.000226, -0.000126, -3.0e-06, 5.0e-06, 0.000176, 0.0005200000000000001, 0.00062, 0.0007199999999999999, 0.0009699999999999999, 0.0012, 0.0017, 0.0022, 0.0028, 0.0037, 0.0045000000000000005, 0.0066, 0.008199999999999999, 0.0091, 0.01], :mode=>"markers+lines", :name=>"nakagawa_dark"}, {:x=>[0.247, 0.254, 0.258, 0.266, 0.271, 0.277, 0.281, 0.286, 0.292, 0.298, 0.322], :y=>[-0.0024500000000000004, -0.0021000000000000003, -0.0019, -0.0016, -0.0014, -0.0011, -0.00091, -0.0007, -0.0005, -0.00026, 0], :mode=>"markers+lines", :name=>"nakagawa_5cm"}, {:x=>[0.24, 0.254, 0.272, 0.286, 0.299, 0.312, 0.32, 0.329, 0.334, 0.35], :y=>[-0.0024, -0.0021000000000000003, -0.0017, -0.0014, -0.0011, -0.0008, -0.0006, -0.0004, -0.00029, 0], :mode=>"markers+lines", :name=>"nakagawa_15cm"}, {:x=>[0.157, 0.174, 0.195, 0.215, 0.23, 0.256, 0.277, 0.291, 0.314], :y=>[-0.0015600000000000002, -0.0014, -0.0012, -0.001, -0.0008, -0.0006, -0.0004, -0.00025, 0], :mode=>"markers+lines", :name=>"nakagawa_25cm"}], @layout={:yaxis=>{:title=>"電流(I)"}, :xaxis=>{:title=>"電圧(V)"}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

# ダイオード?

In [13]:
data = ['sangenya', 'nagura', 'nakagawa'].map do |member|
  ['dark', '5cm', '15cm', '25cm', 'dark_without_resistance'].map do |name|
    file_name = "data/#{member}/#{name}_solar_battery_negative.csv"
    if File.exists?(file_name)
      Daru::DataFrame.from_csv(file_name).sort(['v']).to_h.map { |k, v| [k == 'v' ? 'x' : 'y', v.to_a] }.to_h.merge(name: "#{member}_#{name}", mode: 'markers+lines')
    end
  end
end.flatten
Plotly::Plot.new(data: data, layout: { xaxis: { title: '電圧(V)' }, yaxis: { title: '電流(I)' } }).show
nil

#<Plotly::Offline::HTML:0x007fd99aac2710 @id="f02fb838-4d0c-428c-a1fd-a3f41602e305", @data=[{:x=>[-0.991, -0.911, -0.874, -0.802, -0.774, -0.734, -0.709, -0.614, -0.538, -0.458, -0.35, -0.142, -0.07529999999999999, -0.0346, -0.0008, 0.105, 0.123, 0.165, 0.19, 0.267, 0.311, 0.339, 0.39, 0.418, 0.44, 0.459, 0.486, 0.505, 0.519, 0.538, 0.554, 0.565, 0.582], :y=>[-0.009000000000000001, -0.0068, -0.005900000000000001, -0.0044, -0.0039, -0.0033, -0.0029, -0.0017, -0.00115, -0.000743, -0.000439, -0.00014, -7.3e-05, -3.3e-05, -8.0e-07, 8.0e-07, 9.6e-05, 0.00023, 0.000226, 0.00041, 0.00059, 0.00077, 0.0013000000000000002, 0.0018000000000000002, 0.0023, 0.0029, 0.0039, 0.0048, 0.0055, 0.006500000000000001, 0.0075, 0.0083, 0.0094], :mode=>"markers+lines", :name=>"sangenya_dark"}, {:x=>[0.309, 0.345, 0.36, 0.37, 0.379, 0.384, 0.388, 0.391, 0.392, 0.416], :y=>[-0.003, -0.0023, -0.0018000000000000002, -0.0014, -0.001, -0.0008, -0.0006, -0.0004, -0.00034, 0], :mode=>"markers+lines", :name=>"sangenya_5cm"}, {:x=>[0.144, 0.212, 0.263, 0.298, 0.321, 0.341, 0.355, 0.374], :y=>[-0.00145, -0.0013000000000000002, -0.0011, -0.0009000000000000001, -0.0007, -0.0005, -0.00031, 0], :mode=>"markers+lines", :name=>"sangenya_15cm"}, {:x=>[0.119, 0.203, 0.27, 0.303, 0.328, 0.339, 0.361], :y=>[-0.0012, -0.001, -0.000804, -0.000616, -0.00041, -0.000295, 0], :mode=>"markers+lines", :name=>"sangenya_25cm"}, {}, {:x=>[-1.708, -1.626, -1.4, -1.296, -1.209, -1.111, -1.018, -0.929, -0.814, -0.744, -0.62, -0.499, -0.396, -0.308, -0.115, -0.0594, -0.0292, -0.013800000000000002, -0.0009000000000000001, 0.13, 0.272, 0.342, 0.408, 0.417, 0.479, 0.506, 0.54, 0.572, 0.59, 0.617, 0.651, 0.693], :y=>[-0.010400000000000001, -0.0079, -0.0037, -0.0025, -0.0018000000000000002, -0.0011, -0.00071, -0.00044, -0.00022, -0.00017, -6.8e-05, -2.8e-05, -1.4e-05, -7.4e-06, -2.0e-06, -1.0e-06, -5.0e-07, -3.0e-07, 0, 6.3e-06, 5.7e-05, 0.000173, 0.00041, 0.0006, 0.0013000000000000002, 0.0018000000000000002, 0.0025, 0.0034, 0.0039, 0.0048, 0.006200000000000001, 0.008199999999999999], :mode=>"markers+lines", :name=>"nagura_dark"}, {:x=>[0.18, 0.194, 0.224, 0.247, 0.27, 0.293, 0.31, 0.331, 0.344, 0.35], :y=>[-0.0018000000000000002, -0.0017, -0.0015, -0.0013000000000000002, -0.0011, -0.0009, -0.0007, -0.0005, 0, -0.0003], :mode=>"markers+lines", :name=>"nagura_5cm"}, {:x=>[0.094, 0.154, 0.213, 0.252, 0.278, 0.297, 0.32], :y=>[-0.00094, -0.00086, -0.0007, -0.00055, -0.0004, -0.00026, 0], :mode=>"markers+lines", :name=>"nagura_15cm"}, {:x=>[4.7e-05, 0.121, 0.187, 0.222, 0.249, 0.267, 0.282, 0.339], :y=>[-0.000466, -0.00045, -0.00041, -0.00037, -0.00033, -0.00029, -0.000245, 0], :mode=>"markers+lines", :name=>"nagura_25cm"}, {}, {:x=>[-1.218, -0.617, -0.203, -0.184, -0.159, -0.143, -0.115, -0.103, -0.07790000000000001, -0.0594, -0.028300000000000002, -0.0158, -0.0003, 0.0007, 0.022, 0.06559999999999999, 0.0783, 0.0903, 0.121, 0.146, 0.207, 0.275, 0.348, 0.455, 0.542, 0.704, 0.785, 0.815, 0.844], :y=>[-0.0099, -0.004900000000000001, -0.0016, -0.00147, -0.00127, -0.0011, -0.00092, -0.00082, -0.000618, -0.000472, -0.000226, -0.000126, -3.0e-06, 5.0e-06, 0.000176, 0.0005200000000000001, 0.00062, 0.0007199999999999999, 0.0009699999999999999, 0.0012, 0.0017, 0.0022, 0.0028, 0.0037, 0.0045000000000000005, 0.0066, 0.008199999999999999, 0.0091, 0.01], :mode=>"markers+lines", :name=>"nakagawa_dark"}, {:x=>[0.247, 0.254, 0.258, 0.266, 0.271, 0.277, 0.281, 0.286, 0.292, 0.298, 0.322], :y=>[-0.0024500000000000004, -0.0021000000000000003, -0.0019, -0.0016, -0.0014, -0.0011, -0.00091, -0.0007, -0.0005, -0.00026, 0], :mode=>"markers+lines", :name=>"nakagawa_5cm"}, {:x=>[0.24, 0.254, 0.272, 0.286, 0.299, 0.312, 0.32, 0.329, 0.334, 0.35], :y=>[-0.0024, -0.0021000000000000003, -0.0017, -0.0014, -0.0011, -0.0008, -0.0006, -0.0004, -0.00029, 0], :mode=>"markers+lines", :name=>"nakagawa_15cm"}, {:x=>[0.157, 0.174, 0.195, 0.215, 0.23, 0.256, 0.277, 0.291, 0.314], :y=>[-0.0015600000000000002, -0.0014, -0.0012, -0.001, -

# 効率

In [18]:
data = ['sangenya', 'nagura'].map do |member|
  ['5cm', '15cm', '25cm'].map do |name|
    file_name = "data/#{member}/#{name}_solar_battery_negative.csv"
    ["#{member}_#{name}", Daru::DataFrame.from_csv(file_name).sort(['v'])]
  end.to_h
end.flatten
data = data.first.merge data[1]

data.each do |name, df|
  voltage, current, power = df.map_rows { |row| [row['v'], row['i'], row['i'].abs * row['v'].abs] }.max_by { |v, i, power| power }
  puts "#{name} => #{'%1.2e' % (power * 1000)}(mW), #{current}(I), where V = #{voltage}(V)"
end
nil

sangenya_5cm => 9.27e-01(mW), -0.003(I), where V = 0.309(V)
sangenya_15cm => 2.89e-01(mW), -0.0011(I), where V = 0.263(V)
sangenya_25cm => 2.17e-01(mW), -0.000804(I), where V = 0.27(V)
nagura_5cm => 3.36e-01(mW), -0.0015(I), where V = 0.224(V)
nagura_15cm => 1.49e-01(mW), -0.0007(I), where V = 0.213(V)
nagura_25cm => 8.22e-02(mW), -0.00033(I), where V = 0.249(V)


# n

In [14]:
sangenya_df = Daru::DataFrame.from_csv('data/sangenya/dark_solar_battery_negative.csv').sort(['v'])
sangenya_hash = sangenya_df.to_h.map { |k, v| [k == 'v' ? 'x' : 'y', v.to_a] }.to_h.merge(name: "sangenya_dark", mode: 'markers+lines')
nagura_df = Daru::DataFrame.from_csv('data/nagura/dark_solar_battery_negative.csv').sort(['v'])
nagura_hash = nagura_df.to_h.map { |k, v| [k == 'v' ? 'x' : 'y', v.to_a] }.to_h.merge(name: "nagura_dark", mode: 'markers+lines')

sangenya_to_line = sangenya_df.filter_rows { |row| row['v'] > 0.14 }
sangenya_fit = ExpFitting.new(sangenya_to_line['v'].to_a, sangenya_to_line['i'].to_a)
sangenya_fit.fit

nagura_to_line = nagura_df.filter_rows { |row| row['v'] > 0.1 && row['v'] < 0.58 }
nagura_fit = ExpFitting.new(nagura_to_line['v'].to_a, nagura_to_line['i'].to_a)
nagura_fit.fit

Plotly::Plot.new(data: [sangenya_hash, nagura_hash, sangenya_fit.points(0, 1), nagura_fit.points(0, 1)], layout: { xaxis: { title: '電圧(V)' }, yaxis: { title: '電流(I)', type: :log } }).show
nil

#<Plotly::Offline::HTML:0x007fd99a88a808 @id="e3f9cee7-acac-42a5-8f54-9049f3ed9c29", @data=[{:x=>[-0.991, -0.911, -0.874, -0.802, -0.774, -0.734, -0.709, -0.614, -0.538, -0.458, -0.35, -0.142, -0.07529999999999999, -0.0346, -0.0008, 0.105, 0.123, 0.165, 0.19, 0.267, 0.311, 0.339, 0.39, 0.418, 0.44, 0.459, 0.486, 0.505, 0.519, 0.538, 0.554, 0.565, 0.582], :y=>[-0.009000000000000001, -0.0068, -0.005900000000000001, -0.0044, -0.0039, -0.0033, -0.0029, -0.0017, -0.00115, -0.000743, -0.000439, -0.00014, -7.3e-05, -3.3e-05, -8.0e-07, 8.0e-07, 9.6e-05, 0.00023, 0.000226, 0.00041, 0.00059, 0.00077, 0.0013000000000000002, 0.0018000000000000002, 0.0023, 0.0029, 0.0039, 0.0048, 0.0055, 0.006500000000000001, 0.0075, 0.0083, 0.0094], :mode=>"markers+lines", :name=>"sangenya_dark"}, {:x=>[-1.708, -1.626, -1.4, -1.296, -1.209, -1.111, -1.018, -0.929, -0.814, -0.744, -0.62, -0.499, -0.396, -0.308, -0.115, -0.0594, -0.0292, -0.013800000000000002, -0.0009000000000000001, 0.13, 0.272, 0.342, 0.408, 0.417, 0.479, 0.506, 0.54, 0.572, 0.59, 0.617, 0.651, 0.693], :y=>[-0.010400000000000001, -0.0079, -0.0037, -0.0025, -0.0018000000000000002, -0.0011, -0.00071, -0.00044, -0.00022, -0.00017, -6.8e-05, -2.8e-05, -1.4e-05, -7.4e-06, -2.0e-06, -1.0e-06, -5.0e-07, -3.0e-07, 0, 6.3e-06, 5.7e-05, 0.000173, 0.00041, 0.0006, 0.0013000000000000002, 0.0018000000000000002, 0.0025, 0.0034, 0.0039, 0.0048, 0.006200000000000001, 0.008199999999999999], :mode=>"markers+lines", :name=>"nagura_dark"}, {:x=>[0, 1], :y=>[3.496088566606789e-05, 0.5204962071065878], :name=>"log y = 9.61e+00 x - 1.03e+01"}, {:x=>[0, 1], :y=>[1.12438847148944e-06, 2.1453111538025276], :name=>"log y = 1.45e+01 x - 1.37e+01"}], @layout={:yaxis=>{:title=>"電流(I)", :type=>:log}, :xaxis=>{:title=>"電圧(V)"}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>